<a href="https://colab.research.google.com/github/Warspyt/PC_Python_2025II/blob/main/clase__18/Web_Requests_and_Libs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Programación de Computadores en Python
## Peticiones Web y Librerías Útiles de Python
#### Universidad Nacional de Colombia

**Objetivos:**
- Realizar peticiones HTTP seguras y eficientes con `requests`.
- Usar librerías estándar (`os`, `pathlib`, `datetime`, `random`, `json`, `csv`, `subprocess`) en flujos reales.
- Implementar patrones de reintento, streaming, autenticación con tokens y clientes API simples.
- Practicar con ejercicios y proyectos dentro del notebook.

---

### Contenido detallado
1. Introducción a HTTP/REST y herramientas
2. `requests`: GET/POST/PUT/DELETE, params, headers, timeouts
3. Manejo de respuestas: JSON, texto, streams, archivos
4. Sesiones, cookies y cabeceras persistentes
5. Errores, timeouts, reintentos, backoff y rate-limiting
6. Autenticación básica y tokens (conceptos y uso seguro)
7. Librerías estándar prácticas: `os`, `pathlib`, `datetime`, `time`, `random`, `json`, `csv`, `subprocess`
8. Ejemplos combinados (descarga, parseo, guardado)
9. Ejercicios cortos y dos proyectos largos
10. Tips, recomendaciones y recursos


## 🔧 Cómo usar este notebook

- Ejecuta las celdas con `Shift+Enter` en Colab/Jupyter.
- Si trabajas en Colab y necesitas subir archivos, usa el panel lateral o `files.upload()`.
- Las celdas que tocan internet usan APIs públicas (jsonplaceholder, httpbin). Si no tienes internet, salta esas celdas o usa las de simulación.


### 0️⃣ Instalación e imports

Si `requests` no está instalado en tu entorno, ejecuta:

```bash
!pip install requests
```


In [ ]:
import requests
import os
from pathlib import Path
import datetime
import time
import random
import json
import csv
import subprocess

print('Imports listos. requests version:', getattr(requests, '__version__', 'n/a'))

Imports listos. requests version: 2.32.4


# 1️⃣ Introducción a HTTP y REST

- Cliente-servidor, endpoints, recursos.
- Verbos HTTP: `GET`, `POST`, `PUT`, `DELETE`, `PATCH`, `HEAD`.
- Status codes: 200, 201, 204, 400, 401, 403, 404, 429, 500.
- REST y formato JSON como estándar de intercambio.

Ejemplo práctico: usaremos https://jsonplaceholder.typicode.com como API de pruebas.


## 2️⃣ `requests` — GET/POST básicos, params y headers

In [ ]:
# GET simple a API pública
url = 'https://jsonplaceholder.typicode.com/posts/1'
resp = requests.get(url, timeout=5)
print('status:', resp.status_code)
try:
    print('keys:', list(resp.json().keys()))
except Exception as e:
    print('no JSON:', e)


status: 200
keys: ['userId', 'id', 'title', 'body']


### Parámetros, query strings y headers
Usa `params=` para query strings y `headers=` para cabeceras (ej. `User-Agent`).

In [ ]:
url = 'https://jsonplaceholder.typicode.com/comments'
params = {'postId': 1}
headers = {'User-Agent': 'CursoPython/1.0'}
resp = requests.get(url, params=params, headers=headers, timeout=5)
print('status:', resp.status_code, '-> comments count:', len(resp.json()))


status: 200 -> comments count: 5


### POST y envío de JSON

In [ ]:
url = 'https://jsonplaceholder.typicode.com/posts'
payload = {'title':'Hola','body':'Contenido','userId': 1}
resp = requests.post(url, json=payload, timeout=5)
print('status:', resp.status_code)
print('resp id (simulado):', resp.json().get('id'))


status: 201
resp id (simulado): 101


# 3️⃣ Manejo de errores, timeouts y reintentos

- Usa `timeout` para evitar bloqueos.
- Maneja excepciones: `requests.exceptions.Timeout`, `ConnectionError`, `HTTPError`.
- Reintentos con `HTTPAdapter` o bibliotecas como `tenacity`.

In [ ]:
from requests.exceptions import Timeout, RequestException
try:
    resp = requests.get('https://httpbin.org/delay/3', timeout=1)
    resp.raise_for_status()
    print('OK')
except Timeout:
    print('Timeout ocurrido (esperado si timeout < delay)')
except RequestException as e:
    print('Request error:', type(e), e)


Request error: <class 'requests.exceptions.HTTPError'> 503 Server Error: Service Temporarily Unavailable for url: https://httpbin.org/delay/3


## 4️⃣ Sessions y persistencia de headers/cookies

In [ ]:
s = requests.Session()
s.headers.update({'User-Agent':'MiCliente/1.0'})
resp1 = s.get('https://jsonplaceholder.typicode.com/posts/1')
resp2 = s.get('https://jsonplaceholder.typicode.com/posts/2')
print('status1', resp1.status_code, 'status2', resp2.status_code)
print('session headers sample:', dict(list(s.headers.items())[:3]))


status1 200 status2 200
session headers sample: {'User-Agent': 'MiCliente/1.0', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Accept': '*/*'}


## Descarga de archivos y streaming
Usa `stream=True` para descargar archivos grandes por bloques y evitar cargar todo en memoria.

In [ ]:
from pathlib import Path
url = 'https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf'
local = 'dummy.pdf'
with requests.get(url, stream=True, timeout=10) as r:
    r.raise_for_status()
    with open(local,'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
print('Descargado', local, 'size bytes=', Path(local).stat().st_size)


Descargado dummy.pdf size bytes= 13264


# 5️⃣ Autenticación y seguridad (conceptos)

- Básica: `auth=('user','pass')`.
- Tokens Bearer en headers: `Authorization: Bearer <TOKEN>`.
- OAuth y flows: usa librerías específicas para implementarlo.
- Importante: no poner claves en el código; usar variables de entorno (`os.environ`).


## 6️⃣ Rate limiting y backoff

- Respeta `Retry-After`.
- Usa backoff exponencial con jitter para reintentos.
- Limita número de intentos y maneja 429 adecuadamente.


In [ ]:
import time, random

def get_with_retries(url, max_retries=4):
    delay = 1
    for attempt in range(1, max_retries+1):
        try:
            r = requests.get(url, timeout=5)
            r.raise_for_status()
            return r
        except requests.RequestException as e:
            print(f'Attempt {attempt} failed:', e)
            if attempt == max_retries:
                raise
            sleep_time = delay * (2 ** (attempt-1)) + random.random()
            print(f'Waiting {sleep_time:.2f}s before retry')
            time.sleep(sleep_time)

# prueba
try:
    r = get_with_retries('https://jsonplaceholder.typicode.com/posts/1')
    print('OK after retries')
except Exception as e:
    print('Failed finally:', e)


OK after retries


# 7️⃣ Librerías estándar útiles — `os`, `pathlib`, `datetime`, `random`, `json`, `csv`, `subprocess`

In [ ]:
from pathlib import Path
p = Path('ejemplo_dir')
p.mkdir(exist_ok=True)
(p / 'archivo.txt').write_text('Contenido de prueba', encoding='utf-8')
print('List dir:', [x.name for x in p.iterdir()])


List dir: ['archivo.txt']


In [ ]:
import datetime
now = datetime.datetime.now()
print('Now:', now.isoformat())
print('Timestamp:', now.timestamp())


Now: 2025-11-24T15:21:54.349398
Timestamp: 1763997714.349398


In [ ]:
import random
random.seed(42)
print('randint 0-10:', random.randint(0,10))
print('sample 5:', random.sample(range(100),5))


randint 0-10: 10
sample 5: [14, 3, 94, 35, 31]


In [ ]:
import json, csv
obj = {'a':1,'b':[1,2,3]}
with open('ejemplo.json','w',encoding='utf-8') as f:
    json.dump(obj,f,ensure_ascii=False,indent=2)
print('ejemplo.json creado')
with open('ejemplo.csv','w',newline='',encoding='utf-8') as f:
    w = csv.writer(f); w.writerow(['id','valor']); w.writerow([1,'x'])
print('ejemplo.csv creado')


ejemplo.json creado
ejemplo.csv creado


In [ ]:
import subprocess
try:
    res = subprocess.run(['echo','Hola desde subprocess'], capture_output=True, text=True)
    print('stdout:', res.stdout.strip())
except Exception as e:
    print('subprocess error:', e)


stdout: Hola desde subprocess


# 8️⃣ Ejemplos combinados prácticos

- Descargar JSON y guardarlo.
- Consulta periódica con backoff y guardado incremental.
- Cliente API simple con token opcional desde entorno.

In [ ]:
url = 'https://jsonplaceholder.typicode.com/posts'
r = requests.get(url, timeout=5)
data = r.json()
with open('posts.json','w',encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
print('Guardados', len(data), 'posts en posts.json')


Guardados 100 posts en posts.json


In [ ]:
from datetime import datetime

def consulta_periodica(url, rounds=2):
    for i in range(rounds):
        try:
            r = get_with_retries(url, max_retries=3)
            obj = r.json()
            filename = f'results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
            with open(filename,'w',encoding='utf-8') as f:
                json.dump(obj, f, ensure_ascii=False, indent=2)
            print('Guardado en', filename)
        except Exception as e:
            print('Fallo en round', i+1, e)
        time.sleep(1 + random.random())

consulta_periodica('https://jsonplaceholder.typicode.com/posts/1', rounds=2)


Guardado en results_20251124_152212.json
Guardado en results_20251124_152213.json


# 9️⃣ Ejercicios prácticos

Cada ejercicio tiene esqueleto y solución. Intenta resolver antes de mirar la solución.


## Ejercicio 1 — GET y guardar CSV (corto)

Lee `/users` en jsonplaceholder, guarda `users.csv` con id,name,email.

In [ ]:
import requests, csv

def ejercicio1():
    url = 'https://jsonplaceholder.typicode.com/users'
    # TODO: GET, parse JSON, escribir CSV
    pass

# Descomenta para probar
# print(ejercicio1())


In [ ]:
def solucion_ej1():
    url = 'https://jsonplaceholder.typicode.com/users'
    r = requests.get(url, timeout=5); r.raise_for_status()
    users = r.json()
    with open('users.csv','w',newline='',encoding='utf-8') as f:
        w = csv.writer(f); w.writerow(['id','name','email'])
        for u in users:
            w.writerow([u['id'], u['name'], u['email']])
    return len(users)

print('usuarios guardados:', solucion_ej1())


## Ejercicio 2 — Session y cookies (medio)

Usa `requests.Session()` para hacer dos peticiones y guardar cookies en `session_cookies.json`.

In [ ]:
import requests, json

def ejercicio2():
    s = requests.Session()
    # TODO: set headers, hacer requests y guardar cookies
    pass


In [ ]:
def solucion_ej2():
    s = requests.Session(); s.headers.update({'User-Agent':'CursoPython/1.0'})
    r = s.get('https://jsonplaceholder.typicode.com/posts/1', timeout=5); r.raise_for_status()
    cookies = s.cookies.get_dict()
    with open('session_cookies.json','w',encoding='utf-8') as f:
        json.dump(cookies,f)
    return cookies

print('cookies guardadas:', solucion_ej2())


## Ejercicio 3 — Descargar y verificar hash (medio)

Descarga un archivo y calcula su SHA256.

In [ ]:
import requests, hashlib

def ejercicio3(url, out='download.bin'):
    # TODO: descargar streaming y calcular sha256
    pass


In [ ]:
def solucion_ej3(url, out='download.bin'):
    h = hashlib.sha256()
    with requests.get(url, stream=True, timeout=10) as r:
        r.raise_for_status()
        with open(out,'wb') as f:
            for chunk in r.iter_content(8192):
                if chunk:
                    f.write(chunk); h.update(chunk)
    return h.hexdigest()

print('sha256:', solucion_ej3('https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf'))


## Proyecto largo — Cliente API con Session y backoff (largo)

Crea `apiclient.py` con una clase que soporte retries y token opcional en variable de entorno.

In [ ]:
from pathlib import Path
import requests, os

class APIClient:
    def __init__(self, base_url):
        self.base = base_url
        self.s = requests.Session()
        token = os.getenv('API_TOKEN')
        if token:
            self.s.headers.update({'Authorization':f'Bearer {token}'})

    def get_posts(self):
        r = get_with_retries(self.base + '/posts')
        return r.json()

    def get_post(self, pid):
        r = get_with_retries(self.base + f'/posts/{pid}')
        return r.json()

client = APIClient('https://jsonplaceholder.typicode.com')
print('posts sample length:', len(client.get_posts()))


## 🔚 Tips, recomendaciones y recursos

- Nunca expongas tokens en el código; usa variables de entorno o vaults.
- Respeta `robots.txt` y límites de uso de APIs.
- Usa `timeout`, reintentos y backoff; controla logs.
- Para scraping, considera `BeautifulSoup` y respeta términos.

**Recursos:**
- Requests docs: https://docs.python-requests.org/
- Tenacity retries: https://github.com/jd/tenacity
- HTTP status: https://httpstatuses.com/
